In [1]:
from arcgis import GIS
from arcgis.features.layer import FeatureLayerCollection

### Anatomy of a Parcel Fabric

A parcel fabric stores a dataset of connected parcels or parcel network. Parcels are composed of polygon features, line features, and point features. Parcel polygons are defined by lines that store COGO dimensions from the recorded, legal document.

A Parcel Fabric consists of:

1. A geodatabase topology
2. A polygon feature class called Records for storing legal document information about a set of parcels.
3. A point feature class representing the physical, coordinated x,y,z locations on the ground.
4. A lines feature class called Connection Lines to represent dimensions between points that are not parcel boundaries.
5. Any number of parcel types

<img align='left' width='300' height='280' src='https://pro.arcgis.com/en/pro-app/latest/help/data/parcel-editing/GUID-2239DE47-1DCF-42ED-B58D-D6C4B99DCA71-web.png' />

A parcel type is composed of a polygon and line feature class and can be extended to contain attributes required by an organization.
A parcel type consists of a pair of feature classes.  
- A polygon feature class representing the area of a parcel.
- A COGO enabled line feature class representing the boundary of a parcel.

The cells below will demonstrate how to access the different parts of a parcel fabric

In [3]:
base_server_url = "https://myserver.domain.com/web_adaptor/rest/services/WashingtonCountyLSA/"
gis = GIS("https://myserver.domain.com/web_adaptor/", "username", "pass.word", verify_cert=False)

# generate the enpoint urls for feature server, version management and parcel fabric
service_endpoints = ["FeatureServer", "ParcelFabricServer", "VersionManagementServer"]                    
service_urls = {url: base_server_url + url for url in service_endpoints}

[v for k, v in service_urls.items()]

['https://myserver.domain.com/web_adaptor/rest/services/WashingtonCountyLSA/FeatureServer',
 'https://myserver.domain.com/web_adaptor/rest/services/WashingtonCountyLSA/ParcelFabricServer',
 'https://myserver.domain.com/web_adaptor/rest/services/WashingtonCountyLSA/VersionManagementServer']

### The Parcel Fabric FeatureLayerCollection

A Parcel Fabric is published as an entire set containing the feature classes and topology described above.  The FeatureLayerCollection (FLC) provides access to these items as well as the properties and capabilities of the feature service itself.

In [4]:
parcel_fabric_flc = FeatureLayerCollection(service_urls["FeatureServer"], gis)
parcel_fabric_flc.properties.capabilities

'Query,Create,Update,Delete,Uploads,Editing,ChangeTracking'

### Parcel Fabric Feature Layers

In [5]:
[f"Layer ID: {layer.id} - Name: {layer.name}" 
    for layer in parcel_fabric_flc.properties.layers 
    if layer.type == "Feature Layer"]

['Layer ID: 1 - Name: Records',
 'Layer ID: 3 - Name: Dirty Areas',
 'Layer ID: 4 - Name: Point Errors',
 'Layer ID: 5 - Name: Line Errors',
 'Layer ID: 6 - Name: Polygon Errors',
 'Layer ID: 7 - Name: Points',
 'Layer ID: 8 - Name: Connection Lines',
 'Layer ID: 10 - Name: Adjustment Points',
 'Layer ID: 11 - Name: Adjustment Lines',
 'Layer ID: 12 - Name: Adjustment Vectors',
 'Layer ID: 14 - Name: Tax_PF_Lines',
 'Layer ID: 15 - Name: Tax_PF',
 'Layer ID: 17 - Name: ConveyanceDivision_PF_Lines',
 'Layer ID: 18 - Name: ConveyanceDivision_PF',
 'Layer ID: 20 - Name: Block_PF_Lines',
 'Layer ID: 21 - Name: Block_PF',
 'Layer ID: 23 - Name: SimultaneousConveyance_PF_Lines',
 'Layer ID: 24 - Name: SimultaneousConveyance_PF']

### The ParcelFabricManager object

The Parcel Fabric Manager is responsible for exposing parcel management capabilities to support a variety of workflows from different clients and systems.


In [7]:
from arcgis.features._parcel import ParcelFabricManager

# Create VMS from parcel fabric FLC
version_management_service = parcel_fabric_flc.versions

with version_management_service.get("admin.generate_fabric_links", "read") as version:
    parcel_fabric_manager = ParcelFabricManager(
        service_urls["ParcelFabricServer"], 
        gis, 
        version, 
        parcel_fabric_flc)

parcel_fabric_manager.properties

{
  "name": "Parcel Fabric Server",
  "type": "Map Server Extension",
  "ParcelFabricLayers": 1
}

#### Access the Parcel Fabric SOE functions

In [8]:
[n for n in dir(parcel_fabric_manager) if not n.startswith("_")]

['analyze_least_squares_adjustment',
 'apply_least_squares_adjustment',
 'assign_to_record',
 'build',
 'change_type',
 'clip',
 'copy_lines_to_parcel_type',
 'create_seeds',
 'delete',
 'divide',
 'duplicate',
 'layer',
 'merge',
 'properties',
 'update_history']

### Documentation

[ArcGIS Python API - Version Manager](https://developers.arcgis.com/python/api-reference/arcgis.features.managers.html#versionmanager)

[ArcGIS Python API - Parcel Fabric Manager](https://developers.arcgis.com/python/api-reference/arcgis.features.managers.html#parcelfabricmanager)

[ArcGIS REST API - VersionManagementServer](https://developers.arcgis.com/rest/services-reference/enterprise/version-management-service.htm)

[ArcGIS REST API - ParcelFabricServer](https://developers.arcgis.com/rest/services-reference/enterprise/overview-of-parcel-fabric-sevices.htm)

[ArcGIS Pro - Branch Versioning Scenarios](https://pro.arcgis.com/en/pro-app/latest/help/data/geodatabases/overview/branch-version-scenarios.htm)